# Imports

In [1]:
from pathlib import Path
import torch
from torch import nn
from torch import optim
from mltrainer import metrics, Trainer, TrainerSettings, ReportTypes
from mltrainer.preprocessors import BasePreprocessor
import mlflow
from hyperopt import fmin, tpe, hp, STATUS_OK, Trials
from hyperopt.pyll import scope
from mads_datasets import DatasetFactoryProvider, DatasetType
from loguru import logger
from datetime import datetime
import seaborn as sns
import matplotlib.pyplot as plt

# Datastreamer

In [2]:
datafactory = DatasetFactoryProvider.create_factory(DatasetType.FASHION)
batchsize = 64
preprocessor = BasePreprocessor()
streamers = datafactory.create_datastreamer(batchsize=batchsize, preprocessor=preprocessor)
train = streamers["train"]
valid = streamers["valid"]
train_streamer = train.stream()
valid_streamer = valid.stream()

2023-12-17 08:36:13.187 | INFO     | mads_datasets.base:download_data:121 - Folder already exists at /home/azureuser/.cache/mads_datasets/fashionmnist
2023-12-17 08:36:13.188 | INFO     | mads_datasets.base:download_data:124 - File already exists at /home/azureuser/.cache/mads_datasets/fashionmnist/fashionmnist.pt


# Model

In [27]:
# Although this program will likely only run on the vm,
# and the vm lacks gpu support, a gpu check is added in case the
# program is ran outside of the vm.

device = "gpu" if torch.cuda.is_available() else "cpu"
print(f"Model training takes place on the {device}")


class CNN(nn.Module):
    def __init__(
        self,
        layers: int,
        filters: int,
        units: int,
        normalization: bool = False,
        dropout: float = 0.0,
        input_size: tuple = (32, 1, 28, 28),
    ):
        super().__init__()

        self.convolutional_blocks = nn.ModuleList()
        for i in range(1, layers):
            first = i == 1
            self.convolutional_blocks.append(
                self._generate_convolutional_layer(filters=filters, normalization=normalization, first=first)
            )

        # In order to solve the potential problem of connecting the image layers
        # with the linear layers, an AvgPool2d is added based on the size of the
        # activationmap from the convolutional layers.
        # Resulting in (batch, activationmaps, 1, 1) before flattening.
        activaton_map_size = self._conv_test(input_size)
        self.aggregated_layer = nn.AvgPool2d(activaton_map_size)

        self.linear_blocks = nn.ModuleList()
        for i in range(1, layers):
            first = i == 1
            last = i == (layers - 1)
            self.linear_blocks.append(
                self._generate_linear_layers(
                    filters=filters,
                    units=units,
                    dropout=dropout,
                    normalization=normalization,
                    first=first,
                    last=last,
                )
            )

    def _generate_convolutional_layer(
        self,
        filters: int, normalization: bool = False, first: bool = False
    ) -> nn.Sequential:
        in_channels = 1 if first else filters
        layers = []
        layers.append(nn.Conv2d(in_channels, filters, kernel_size=3))
        layers.append(nn.BatchNorm2d(filters)) if normalization else None
        layers.append(nn.ReLU())
        layers.append(nn.MaxPool2d(kernel_size=2))
        return nn.Sequential(*layers)

    def _conv_test(self, input_size):
        x = torch.ones(input_size)
        for block in self.convolutional_blocks:
            x = block(x)
        return x.shape[-2:]

    def _generate_linear_layers(
        self,
        filters: int,
        units: int,
        dropout: float = 0.0,
        normalization: bool = False,
        first: bool = False,
        last: bool = False,
    ) -> nn.Sequential:
        in_features = filters if first else units
        out_features = 10 if last else units
        layers = []
        layers.append(nn.Flatten()) if first else None
        layers.append(nn.Dropout1d(dropout)) if dropout != 0.0 and last else None
        layers.append(nn.Linear(in_features, out_features))
        layers.append(nn.BatchNorm1d(units)) if normalization and not last else None
        layers.append(nn.ReLU()) if not last else None
        return nn.Sequential(*layers)

    def forward(self, x):
        for block in self.convolutional_blocks:
            x = block(x)
        x = self.aggregated_layer(x)
        for block in self.linear_blocks:
            x = block(x)
        return x


model = CNN(layers=2, filters=32, units=32, dropout=0.2).to(device)


Model training takes place on the cpu


# Optimizer, loss & accuracy

In [17]:
optimizer = optim.Adam
loss_function = torch.nn.CrossEntropyLoss()
accuracy = metrics.Accuracy()

# MLflow

In [19]:
experiment_path = "MNIST_CNN"
mlflow.set_tracking_uri("sqlite:///mlflow.db")
mlflow.set_experiment(experiment_path)

2023/12/17 11:22:45 INFO mlflow.tracking.fluent: Experiment with name 'MNIST_CNN' does not exist. Creating a new experiment.


<Experiment: artifact_location='/home/azureuser/code/ADS-DeepLearning-project/hyper_parameter_exercise/notebook/mlruns/3', creation_time=1702812165353, experiment_id='3', last_update_time=1702812165353, lifecycle_stage='active', name='MNIST_CNN', tags={}>

In [20]:
model_directory = Path("../../models/mnist").resolve()
if not model_directory.exists():
    model_directory.mkdir(parents=True)
    print(f"Created {model_directory}")

In [24]:
trainer_settings = TrainerSettings(
    epochs=3,
    metrics=[accuracy],
    logdir="modellog",
    train_steps=100,
    valid_steps=100,
    reporttypes=[ReportTypes.MLFLOW]
)

def objective(params):
    with mlflow.start_run():
        mlflow.set_tag("model", "convolutional_network")
        mlflow.set_tag("dev", "tom")
        mlflow.log_params(params)
        mlflow.log_param("batchsize", f"{batchsize}")

        model = CNN(**params)
        trainer = Trainer(
            model=model,
            settings=trainer_settings,
            loss_fn=loss_function,
            optimizer=optimizer,
            traindataloader=train_streamer,
            validdataloader=valid_streamer,
            scheduler=optim.lr_scheduler.ReduceLROnPlateau
        )
        trainer.loop()

        tag = datetime.now().strftime("%Y%m%d-%H%M")
        model_path = model_directory / (tag + "model.pt")
        torch.save(model, model_path)

        mlflow.log_artifact(local_path=model_path, artifact_path="pytorch_models")
        
        return {"loss": trainer.test_loss, "status": STATUS_OK}

In [32]:
search_space = {
    "layers": scope.int(hp.quniform("layers", 2, 5, 1)),
    "units": scope.int(hp.quniform("units", 32, 128, 8)),
    "filters": 32,
}

In [33]:
best_result = fmin(
    fn=objective,
    space=search_space,
    algo=tpe.suggest,
    max_evals=100,
    trials=Trials()
)

  0%|          | 0/3 [00:00<?, ?trial/s, best loss=?]

2023-12-17 11:29:54.267 | INFO     | mltrainer.trainer:dir_add_timestamp:29 - Logging to modellog/20231217-112954
2023-12-17 11:29:55.618 | INFO     | mltrainer.trainer:__init__:65 - Found earlystop_kwargs in settings.Set to None if you dont want earlystopping.
100%|##########| 100/100 [00:02<00:00, 41.18it/s]
2023-12-17 11:29:59.476 | INFO     | mltrainer.trainer:report:171 - Epoch 0 train 2.2901 test 2.2657 metric ['0.0942']
100%|##########| 100/100 [00:02<00:00, 40.64it/s]
2023-12-17 11:30:03.308 | INFO     | mltrainer.trainer:report:171 - Epoch 1 train 2.2346 test 2.2016 metric ['0.1750']
100%|##########| 100/100 [00:02<00:00, 44.99it/s]
2023-12-17 11:30:06.949 | INFO     | mltrainer.trainer:report:171 - Epoch 2 train 2.1595 test 2.1146 metric ['0.2955']
100%|##########| 3/3 [00:11<00:00,  3.78s/it]


 33%|███▎      | 1/3 [00:13<00:26, 13.11s/trial, best loss: 2.1145686292648316]

2023-12-17 11:30:07.181 | INFO     | mltrainer.trainer:dir_add_timestamp:29 - Logging to modellog/20231217-113007
2023-12-17 11:30:07.185 | INFO     | mltrainer.trainer:__init__:65 - Found earlystop_kwargs in settings.Set to None if you dont want earlystopping.
100%|##########| 100/100 [00:02<00:00, 36.01it/s]
2023-12-17 11:30:11.635 | INFO     | mltrainer.trainer:report:171 - Epoch 0 train 2.0936 test 1.7418 metric ['0.4225']
100%|##########| 100/100 [00:02<00:00, 36.67it/s]
2023-12-17 11:30:16.042 | INFO     | mltrainer.trainer:report:171 - Epoch 1 train 1.4797 test 1.2585 metric ['0.4953']
100%|##########| 100/100 [00:02<00:00, 33.56it/s]
2023-12-17 11:30:20.667 | INFO     | mltrainer.trainer:report:171 - Epoch 2 train 1.1268 test 1.2139 metric ['0.5337']
100%|##########| 3/3 [00:13<00:00,  4.49s/it]


 67%|██████▋   | 2/3 [00:26<00:13, 13.47s/trial, best loss: 1.2139101600646973]

2023-12-17 11:30:20.864 | INFO     | mltrainer.trainer:dir_add_timestamp:29 - Logging to modellog/20231217-113020
2023-12-17 11:30:20.866 | INFO     | mltrainer.trainer:__init__:65 - Found earlystop_kwargs in settings.Set to None if you dont want earlystopping.
100%|##########| 100/100 [00:02<00:00, 45.19it/s]
2023-12-17 11:30:24.458 | INFO     | mltrainer.trainer:report:171 - Epoch 0 train 2.2893 test 2.2611 metric ['0.1119']
100%|##########| 100/100 [00:01<00:00, 53.49it/s]
2023-12-17 11:30:27.660 | INFO     | mltrainer.trainer:report:171 - Epoch 1 train 2.2299 test 2.1879 metric ['0.2352']
100%|##########| 100/100 [00:01<00:00, 50.71it/s]
2023-12-17 11:30:31.146 | INFO     | mltrainer.trainer:report:171 - Epoch 2 train 2.1440 test 2.0925 metric ['0.3500']
100%|##########| 3/3 [00:10<00:00,  3.43s/it]


100%|██████████| 3/3 [00:37<00:00, 12.44s/trial, best loss: 1.2139101600646973]


In [34]:
best_result

{'layers': 3.0, 'units': 88.0}

In [1]:
def read_mlflow_data(experiment_name):
    runs = mlflow.search_runs(experiment_ids=[experiment_name])
    return runs

mlflow_data = read_mlflow_data('2')

def create_heatmap_data(mlflow_data):
    heatmap_data = mlflow_data[['params.filters', 'params.units1', 'metrics.Loss/test']]
    return heatmap_data

def plot_heatmap(heatmap_data):
    heatmap_pivot = heatmap_data.pivot(index='params.filters', columns='params.units1', values='metrics.Loss/test')
    sns.heatmap(heatmap_pivot, annot=True, fmt=".4f", cmap="YlGnBu")
    plt.xlabel('Units')
    plt.ylabel('Dropout')
    plt.title('Test loss as a function of Units vs Dropout')
    plt.show()

heatmap_data = create_heatmap_data(mlflow_data)
plot_heatmap(heatmap_data)

NameError: name 'mlflow' is not defined